In [71]:
import pandas as pd

def isNaN(x):
    return x != x

data1 = pd.read_csv("IDCJAC0001_070014_Data1.csv")
data1.to_csv("old_rain.csv")
data2 = pd.read_csv("IDCJAC0002_070014_Data1.csv")
data2.to_csv("old_temp.csv")
data3 = pd.read_csv("IDCJAC0001_070351_Data1.csv")
data3.to_csv("new_rain.csv")
data4 = pd.read_csv("IDCJAC0002_070351_Data1.csv")
data4.to_csv("new_temp.csv")

climate_data = pd.DataFrame(columns={"rainfall", "max_temp"})

for i, row in data1.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "rainfall"] = row["Monthly Precipitation Total (millimetres)"]

for i, row in data2.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "max_temp"] = row["Mean maximum temperature (°C)"]

for i, row in data3.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "rainfall"] = row["Monthly Precipitation Total (millimetres)"]

for i, row in data4.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "max_temp"] = row["Mean maximum temperature (°C)"]

climate_data[climate_data.index > pd.to_datetime("1970-12", format="%Y-%m")].to_csv("climate_data.csv", index_label="date")

In [72]:
soi_data = pd.read_csv("soi_monthly.txt", header=None, names=["month", "soi"])
soi_data["month"] = pd.to_datetime(soi_data["month"], format="%Y%m")

soi_mean = []
for i in soi_data.index:
    soi_mean.append(round(soi_data.loc[i - 5:i].soi.mean(), 1))

soi_data["mean"] = soi_mean

soi_data[soi_data["month"] > pd.to_datetime("1970-12", format="%Y-%m")].to_csv("soi_data.csv", index=False)

In [ ]:
import pandas as pd

def isNaN(x):
    return x != x

data1 = pd.read_csv("IDCJAC0009_070014_1800_Data.csv")
data1 = data1[(data1["Quality"] == "Y") | (data1["Quality"] == "N")]
data1.to_csv("old_rain.csv")
data2 = pd.read_csv("IDCJAC0010_070014_1800_Data.csv")
data2 = data2[(data2["Quality"] == "Y") | (data2["Quality"] == "N")]
data2.to_csv("old_temp.csv")
data3 = pd.read_csv("IDCJAC0009_070351_1800_Data.csv")
data3 = data3[(data3["Quality"] == "Y") | (data3["Quality"] == "N")]
data3.to_csv("new_rain.csv")
data4 = pd.read_csv("IDCJAC0010_070351_1800_Data.csv")
data4 = data4[(data4["Quality"] == "Y") | (data4["Quality"] == "N")]
data4.to_csv("new_temp.csv")

climate_data = pd.DataFrame(columns={"rainfall", "max_temp"})

for i, row in data1.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m-%d")
    climate_data.loc[date, "rainfall"] = row["Rainfall amount (millimetres)"]
    if i % 1000 == 0:
        print(f"{(i + 1) / (len(data1)) * 100: .1f}% of old rain data processed")

for i, row in data2.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m-%d")
    climate_data.loc[date, "max_temp"] = row["Maximum temperature (Degree C)"]
    if i % 1000 == 0:
        print(f"{(i + 1) / (len(data2)) * 100: .1f}% of old temperature data processed")

for i, row in data3.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m-%d")
    climate_data.loc[date, "rainfall"] = row["Rainfall amount (millimetres)"]
    if i % 1000 == 0:
        print(f"{(i + 1) / (len(data3)) * 100: .1f}% of new rainfall data processed")
        
for i, row in data4.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}-{str(row.Day).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m-%d")
    climate_data.loc[date, "max_temp"] = row["Maximum temperature (Degree C)"]
    if i % 1000 == 0:
        print(f"{(i + 1) / (len(data4)) * 100: .1f}% of new temperature data processed")

climate_data.sort_index(ascending=True, inplace=True)
        
date_start = climate_data.index.min()
date_end =  climate_data.index.max()
one_year = pd.Timedelta(364, unit="days")
one_day = pd.Timedelta(1, unit="days")

clean_data = climate_data

rainfall_annual = []
temperature_annual = []

for i, date in enumerate(climate_data.index, start=1):    
    if date - one_year < date_start:
        range_start = date_start
        range_end = date_start + one_year
    else:
        range_start = date - one_year
        range_end = date
    
    if isNaN(climate_data.loc[date].max_temp):
        if date == range_end:
            imputed_temp = climate_data.loc[date - one_day].max_temp
        elif isNaN(climate_data.loc[date + one_day].max_temp):
            imputed_temp = climate_data.loc[date - one_day].max_temp
        else:
            imputed_temp = (climate_data.loc[date - one_day].max_temp + climate_data.loc[date + one_day].max_temp) / 2
        print(f"imputing missing temperature value for {date:%d %b %Y}: {imputed_temp}")
        clean_data.loc[date].max_temp = imputed_temp
    
    if isNaN(climate_data.loc[date].rainfall):
        if isNaN(climate_data.loc[date + one_day].rainfall):
            imputed_rainfall = climate_data.loc[date - one_day].rainfall
        else:
            imputed_rainfall = (climate_data.loc[date - one_day].rainfall + climate_data.loc[date + one_day].rainfall) / 2
        print(f"imputing missing rainfall value for {date:%d %b %Y}: {imputed_rainfall}")
        clean_data.loc[date].rainfall = imputed_rainfall

    filtered_data = clean_data.loc[(clean_data.index >= range_start) & (clean_data.index <= range_end)]
    
    rainfall_annual.append(filtered_data.rainfall.sum())
    temperature_annual.append(filtered_data.max_temp.mean())
    
    if i % 1000 == 0:
        print(f"{i / (len(climate_data)) * 100: .1f}% processed")

clean_data["rainfall_annual"] = rainfall_annual
clean_data["temp_annual"] = temperature_annual
clean_data.sort_index(ascending=True, inplace=True)
clean_data["rainfall_annual"] = clean_data["rainfall_annual"].round(1)
clean_data["temp_annual"] = clean_data["temp_annual"].round(1)
clean_data.to_csv("climate_data.csv", index_label="date")
clean_data